In [530]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapefile
import json
from matplotlib.collections import PatchCollection
from IPython.core.pylabtools import figsize
from matplotlib.path import Path
import matplotlib.patches as Patches
from matplotlib import cm,colors


Using matplotlib backend: Qt5Agg


In [11]:
# 导入Shapefile
sf = shapefile.Reader(r'C:\Users\92143\Desktop\gadm36_USA_shp\gadm36_USA_1.shp')

In [495]:
# sf.record(i)[3]           # 州名
# sf.shape(i).points        # 该州边界点集
# sf.shape(i).parts         # 每一个闭环的第一个点在points中的序号

In [531]:
# 分别绘制每个州的边界，会发现很多州由多个闭环构成
# 州的轮廓往往由一个大闭环描绘，其他小的闭环则是细节
for i in range(0,51):
    po = list(zip(*sf.shape(i).points))
    pa = sf.shape(i).parts
    plt.subplot(5,11,i+1)
    if len(pa)==1:
        plt.plot(po[0],po[1])
    else:
        for j in range(len(pa)-1):
            plt.plot(po[0][pa[j]:pa[j+1]],po[1][pa[j]:pa[j+1]])
        plt.plot(po[0][pa[-1]:],po[1][pa[-1]:])
plt.show

<function matplotlib.pyplot.show(*args, **kw)>

In [184]:
# 从shapefile中取出50个州的点集
# 识别出每个点集中最大的闭环，舍弃其他小闭环
# 去掉了阿拉斯加和夏威夷州，留下本土48州

lis = list(range(0,51))
lis.remove(1)
lis.remove(11)
points=[]
for i in lis:
    po=sf.shape(i).points
    pa=sf.shape(i).parts
    pa.append(len(po))
    diff = [(pa[j+1]-pa[j]) for j in range(len(pa)-1)]
    m=diff.index(max(diff))
    points.append(sf.shape(i).points[pa[m]:(pa[m+1]-1)])

# 手动修改一下出现了特殊情况的纽约州   
points[30]=sf.shape(32).points[7966:12216]  

# 使环首尾闭合
for i in range(len(points)):
    points[i].append(points[i][0])

In [506]:
# 画图查看，发现已经可以画出简化的美国地图
fig,ax=plt.subplots()
for i in range(49):
    po = list(zip(*points[i]))
    plt.plot(po[0],po[1])
plt.show

<function matplotlib.pyplot.show(*args, **kw)>

In [504]:
# 用于查看各州剩余边界点的个数
info_new = pd.DataFrame(columns=['state','points'])
j=0
for i in lis:
    ap = pd.DataFrame([[sf.record(i)[3],len(points[j])]],columns=['state','points'])
    info_new = info_new.append(ap,ignore_index=True)
    j += 1
info_new

,state,points
0,Alabama,8950
1,Arizona,1655
2,Arkansas,1671
3,California,61365
4,Colorado,327
5,Connecticut,10174
6,Delaware,6824
7,District of Columbia,163
8,Florida,132275
9,Georgia,12506


In [249]:
# 用“射线法”判断某采样点属于哪一个州

# 遍历每个州的每条边，判断其是否与过给定点向右的水平射线相交
# 记录射线与该州的边的交点总数
# 若为奇数，点在内部；若为偶数，点在外部
# 返回点所在州的编号，若不在任何州内，返回99
def isWithinPoly(poi,poly):
    n=0            
    for epoly in poly: 
        sinsc=0
        for i in range(len(epoly)-1): #[0,len-1]
            s_poi=epoly[i]
            e_poi=epoly[i+1]
            if isIntersectPossible(poi,s_poi,e_poi):
                sinsc+=1 
        if sinsc%2==1:
            return n    # 返回所在州编号
        n += 1
    return 99           # 不在任何州


# 排除射线与州的某边不可能相交的情况
def isIntersectPossible(poi,s_poi,e_poi): 
    if s_poi[1]==e_poi[1]:                   # 边是一个点
        return False
    if s_poi[1]>poi[1] and e_poi[1]>poi[1]:  # 边在射线上方
        return False
    if s_poi[1]<poi[1] and e_poi[1]<poi[1]:  # 边在射线下方
        return False
    if s_poi[1]==poi[1] and e_poi[1]>poi[1]: # 交点为下端点
        return False
    if e_poi[1]==poi[1] and s_poi[1]>poi[1]: # 交点为下端点
        return False
    if s_poi[0]<poi[0] and e_poi[0]<poi[0]:  # 边在射线左侧
        return False

    xseg=e_poi[0]-(e_poi[0]-s_poi[0])*(e_poi[1]-poi[1])/(e_poi[1]-s_poi[1])
    if xseg<poi[0]:                          # 交点在射线起点的左侧
        return False
    return True                              # 有可能相交

In [213]:
names = list(info_few['state'])

In [ ]:
# 遍历采样点，标注所在州
x = np.linspace(-130,-60,150)
y = np.linspace(23,51,60)

ss2=[]
for i in x:
    for j in y:
        ss2.append([i,j])

for i in range(len(ss2)):
    n = isWithinPoly(ss2[i],points)
    ss2[i].append(n)
    print('done ',i,' ')

In [309]:
# 将标记好的150*60个采样点导出到json文件
dic3={}
dic3['states']=names
dic3['samples']=ss2
json_str = json.dumps(dic3, indent=4)
with open('samples_150x60.json', 'w') as json_file:
    json_file.write(json_str)

In [314]:
# 更稀疏的50*20采样版本，便于计算
re = np.array(ss2)
r=re.reshape(150,60,3)
new_sample=[]
for i in range(0,150,3):
    for j in range(0,60,3):
        new_sample.append(list(r[i,j]))
dic4={}
dic4['states']=names
dic4['samples']=new_sample
json_str = json.dumps(dic4, indent=4)
with open('samples_25x10.json', 'w') as json_file:
    json_file.write(json_str)

In [369]:
# 画出采样点
figsize(5,2) # 设置 figsize
z=list(zip(*new_sample))
fig,ax=plt.subplots()
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
plt.margins(0,0)
plt.xlim=((-13,-60))
plt.ylim=((23,51))
for i in range(49):
    po = list(zip(*points[i]))
    plt.plot(po[0],po[1],color='black',lw=0.5)
plt.scatter(z[0], z[1], c=z[2],cmap='GnBu',s=1)
#plt.savefig('t.png', dpi=300)

In [529]:
# 根据点集，使用PathPatch构造自定义多边形，并上色
# 这里按照各州人口数量渲染颜色

pop=np.array([224654,355311,129812,3018337,376994,279653,76537,142502,1059144,601503,78640,879947,371629,192608,169558,211621,255492,
              65349,417776,575635,537087,374920,115749,323287,49635,124742,168752,86046,634721,101452,1701399,575605,55657,689139,
              203250,243085,803307,61341,234367,52544,373663,1818585,180862,34154,544348,576624,79168,342470,39899])
ma=max(pop)
mi=min(pop)
pop=(pop-mi)/(ma-mi)*255

figsize(5,2) # 设置 figsize
fig,ax=plt.subplots()
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
plt.margins(0,0)
c = cm.get_cmap('plasma')
ind=0
for i in points:
    verts=i
    codes=[Path.MOVETO]+[Path.LINETO]*(len(verts)-2)+[Path.CLOSEPOLY]
    path = Path(verts, codes)
    
    value=int(pop[ind])+55
    col=np.array(c.colors[value])
    
    patch = Patches.PathPatch(path, facecolor=col, lw=0.2,edgecolor='white')
    ax.add_patch(patch)
    ind+=1

ax.set_xlim(-130,-60)
ax.set_ylim(23,51)
plt.show()
plt.savefig('xuanran9.png', dpi=300)

In [494]:
# 至此，cartogram的前期准备工作完成了
# 我们获得了按照所属州进行了标记的采样点，还获得了渲染过色彩的美国地图
# 接下来以这张彩色地图为基础，将作为变形依据的指标根据州映射至各采样点
# 应用变形算法，就能获得最终的cartogram